# 2_Transform_data

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Import user defined libraries
import sys
sys.path.append("../src")
from functions import PhaNNS_thresholds, phageRBP_thresholds, NN_thresholds, random_concatenation

## 1. Load clean data

In [2]:
CLEAN_PATH = os.path.join('..', 'data', 'interim', 'clean')

if not os.path.exists(CLEAN_PATH):
    raise Exception('Clean data path does not exist. Did you run the notebook 01_clean_data.ipynb?')

# Import pickle clean data
receptors = pd.read_pickle(os.path.join(CLEAN_PATH, '1_receptors.pkl'))
protein_NN = pd.read_pickle(os.path.join(CLEAN_PATH, '1_proteins.pkl'))
basel_proteome = pd.read_pickle(os.path.join(CLEAN_PATH, '1_basel_proteome.pkl'))
k12_proteome = pd.read_pickle(os.path.join(CLEAN_PATH, '1_k12_proteome.pkl'))


In [3]:
# Display all dataframes
print('Receptors:')
display(receptors.head())
print('Protein NN:')
display(protein_NN.head())
print('Basel proteome:')
display(basel_proteome.head())
print('K12 proteome:')
display(k12_proteome.head())

Receptors:


,bas,genus,phage,morphotype,closest_relative,primary_receptor,terminal_receptor,receptor
0,Bas01,Rtpvirus,Escherichia phage AugustePiccard,siphovirus,RTP (AM156909.1),LPS / O-antigen?,LptD,n.a.
1,Bas02,Guelphvirus,Escherichia phage JeanPiccard,siphovirus,CEB_EC3a (NC_047812.1),LPS / O-antigen?,LptD,n.a.
2,Bas03,Guelphvirus,Escherichia phage JulesPiccard,siphovirus,CEB_EC3a (NC_047812.1),LPS / O-antigen?,FhuA,n.a.
3,Bas04,Warwickvirus,Escherichia phage FritzSarasin,siphovirus,tonnikala (NC_049817.1),LPS / O-antigen?,BtuB,n.a.
4,Bas05,Warwickvirus,Escherichia phage PeterMerian,siphovirus,XY3 (MN781674.1),LPS / O-antigen?,FhuA,n.a.


Protein NN:


,seqID,major_capsid,minor_capsid,baseplate,major_tail,minor_tail,portal,tail_fiber,tail_shaft,collar,HTJ,other,confidence,PhageRBPdetect_prediction,PhageRBPdetect_score,ESM_based_label,ESM_based_fiber_prediction
0,lcl|MZ501051.1_prot_QXV76132.1_1,0.22,0.84,0.18,0.21,0.34,0.85,0.49,0.24,0.46,0.38,5.79,0.98,0,2.403247e-05,other,0
1,lcl|MZ501051.1_prot_QXV76133.1_2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,1.00,0,3.570606e-08,other,0
2,lcl|MZ501051.1_prot_QXV76134.1_3,0.35,1.01,0.71,0.26,1.30,0.97,0.68,0.25,0.24,0.73,3.51,0.97,0,1.817806e-06,other,0
3,lcl|MZ501051.1_prot_QXV76135.1_4,0.01,0.23,0.02,0.01,0.03,0.10,0.09,0.01,0.16,0.03,9.32,1.00,0,1.184212e-05,other,0
4,lcl|MZ501051.1_prot_QXV76136.1_5,0.05,0.07,0.06,0.05,0.13,0.09,0.03,0.07,0.08,0.07,9.28,1.00,0,8.151848e-05,other,0


Basel proteome:


,seqID_phage,locus_tag,protein,protein_id,location,gbkey,sequence,gene,db_xref,frame,partial,exception,bas
0,lcl|MZ501051.1_prot_QXV76132.1_1,bas01_0001,terminase small subunit,QXV76132.1,1..507,CDS,MSKAALKMGEGNFKALYNKKYGDIAMVAINRKYTPEEVFDFAVRYF...,NaN,NaN,NaN,NaN,NaN,Bas01
1,lcl|MZ501051.1_prot_QXV76133.1_2,bas01_0002,hypothetical protein,QXV76133.1,526..621,CDS,MKGFIKLFIWYYLLTSISLCVFMLVVKLWLI,NaN,NaN,NaN,NaN,NaN,Bas01
2,lcl|MZ501051.1_prot_QXV76134.1_3,bas01_0003,terminase large subunit,QXV76134.1,609..2177,CDS,MANLIWEEMTSQEKLAVKAISEHSFEGFLRCWFSITQGERYIPNWH...,NaN,NaN,NaN,NaN,NaN,Bas01
3,lcl|MZ501051.1_prot_QXV76135.1_4,bas01_0004,putative homing endonuclease,QXV76135.1,2321..2716,CDS,MVAGSLSGNGYLHIRIGDRRVKNHLIIWEMHNGRIPEGMEIDHINH...,NaN,NaN,NaN,NaN,NaN,Bas01
4,lcl|MZ501051.1_prot_QXV76136.1_5,bas01_0005,hypothetical protein,QXV76136.1,2788..3168,CDS,MTKKSKAVYLGNTEGEYYGFTVGNEYDVHNYETEDNFIGTFGDDGG...,NaN,NaN,NaN,NaN,NaN,Bas01


K12 proteome:


,seqID_k12,name,OS,OX,GN,PE,SV,sequence
0,sp|A5A616|MGTS_ECOLI,Small protein MgtS,Escherichia coli (strain K12),83333,mgtS,1,1,MLGNMNVFMAVLGIILFSGFLAAYFSHKWDD
1,sp|O32583|THIS_ECOLI,Sulfur carrier protein ThiS,Escherichia coli (strain K12),83333,thiS,1,1,MQILFNDQAMQCAAGQTVHELLEQLDQRQAGAALAINQQIVPREQW...
2,sp|P00350|6PGD_ECOLI,"6-phosphogluconate dehydrogenase, decarboxylat...",Escherichia coli (strain K12),83333,gnd,1,2,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...
3,sp|P00363|FRDA_ECOLI,Fumarate reductase flavoprotein subunit,Escherichia coli (strain K12),83333,frdA,1,3,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...
4,sp|P00370|DHE4_ECOLI,NADP-specific glutamate dehydrogenase,Escherichia coli (strain K12),83333,gdhA,1,1,MDQTYSLESFLNHVQKRDPNQTEFAQAVREVMTTLWPFLEQNPKYR...


## 2. Keep only important columns and rename

### 2.1 Protein receptors
Some of basel receptors are terminal (aka secondary) and some are primary. In both columns there are both LPS and proteins. We only care about proteins, so it might be a good idea to add a column 'receptor_protein' which selects the proteic one from the two. Also, the 'receptor' column has no proteins, so it might be useless for further analysis. \
We will keep only the values which find a match in the k12 gene set.\
NB: some values have a question mark. Since we are aiming fro an as-strict-as-possible labelling, these values are not included

In [4]:
# Get a set of the elements in a df
GN_set = set(k12_proteome['GN'].str.lower().tolist())

# Create a subset that contains only the rows where either primary_receptor or terminal_receptor are in GN_set
receptors = receptors[(receptors['primary_receptor'].str.lower().isin(GN_set)) | (receptors['terminal_receptor'].str.lower().isin(GN_set))]
receptors['receptor_protein'] = np.where(receptors['primary_receptor'].str.lower().isin(GN_set), receptors['primary_receptor'], 
                                      np.where(receptors['terminal_receptor'].str.lower().isin(GN_set), receptors['terminal_receptor'], np.nan))

# keep only useful columns
keep_columns = ['bas', 'phage', 'receptor_protein']
receptors = receptors[keep_columns]

In [5]:
display(receptors.head(2))
display(receptors.shape)

,bas,phage,receptor_protein
0,Bas01,Escherichia phage AugustePiccard,LptD
1,Bas02,Escherichia phage JeanPiccard,LptD


(50, 3)

The number of samples was reduced from 78 to 50

### 2.2 Protein NN
Of the PhaNN classification we will only care about tail_fiber and the confidence score. The rest can be dropped. \
Also, we care only about the binary classification of the ESM-based model

In [6]:
# Rename seqID to seqID_phage
protein_NN = protein_NN.rename(columns={'seqID': 'seqID_phage'})

# Keep only useful columns
keep_columns = ['seqID_phage', 'tail_fiber', 'confidence', 'PhageRBPdetect_prediction', 'PhageRBPdetect_score', 'ESM_based_fiber_prediction']
protein_NN = protein_NN[keep_columns]

In [7]:
display(protein_NN.head(2))
display(protein_NN.shape)

,seqID_phage,tail_fiber,confidence,PhageRBPdetect_prediction,PhageRBPdetect_score,ESM_based_fiber_prediction
0,lcl|MZ501051.1_prot_QXV76132.1_1,0.49,0.98,0,2.403247e-05,0
1,lcl|MZ501051.1_prot_QXV76133.1_2,0.00,1.00,0,3.570606e-08,0


(35416, 6)

### 2.3 Basel proteome

In [8]:
# Rename sequence to sequence_phage
basel_proteome = basel_proteome.rename(columns={'sequence': 'sequence_phage'})

# Keep only useful columns
keep_columns = ['bas', 'seqID_phage', 'protein', 'sequence_phage']
basel_proteome = basel_proteome[keep_columns]

In [9]:
display(basel_proteome.head(2))
display(basel_proteome.shape)

,bas,seqID_phage,protein,sequence_phage
0,Bas01,lcl|MZ501051.1_prot_QXV76132.1_1,terminase small subunit,MSKAALKMGEGNFKALYNKKYGDIAMVAINRKYTPEEVFDFAVRYF...
1,Bas01,lcl|MZ501051.1_prot_QXV76133.1_2,hypothetical protein,MKGFIKLFIWYYLLTSISLCVFMLVVKLWLI


(35416, 4)

### 2.4 k12 proteome

In [10]:
# List columns to keep and columns to rename
keep_columns = ['seqID_k12', 'sequence', 'GN']
change_names = {'sequence': 'sequence_k12',
                'GN': 'GN_k12'}

# Subset receptors_k12 and rename columns
k12_proteome = k12_proteome[keep_columns]
k12_proteome.rename(columns=change_names, inplace=True)

del keep_columns, change_names

In [11]:
display(k12_proteome.head(2))
display(k12_proteome.shape)

,seqID_k12,sequence_k12,GN_k12
0,sp|A5A616|MGTS_ECOLI,MLGNMNVFMAVLGIILFSGFLAAYFSHKWDD,mgtS
1,sp|O32583|THIS_ECOLI,MQILFNDQAMQCAAGQTVHELLEQLDQRQAGAALAINQQIVPREQW...,thiS


(4403, 3)

## 3. Merge receptors and k12_proteome

### 3.1 k12 genes unicity

First of all, let's check for the unicity of the gene names in k12 proteome

In [12]:
# Count occurences in k12_proteome of each GN
GN_count = k12_proteome['GN_k12'].str.lower().value_counts().to_frame()

# Print count of genes with a count of 1 and different than 1
print(f'Number of genes with a count of 1: {len(GN_count.loc[GN_count["count"] == 1])}')
print(f'Number of genes with a count different than 1: {len(GN_count.loc[GN_count["count"] != 1])}')

Number of genes with a count of 1: 4403
Number of genes with a count different than 1: 0


We can confirm that there are no duplicates among the k12 genes

### 3.2 Match receptor and sequence
Let's match each phage in the receptors dataframe with the sequence of the associated receptor, contained in k12_proteome.\
We expect that many entries from receptors could match to the same gene in k12_proteome, and not viceversa. \
We validate this using validate='many_to_one' during the merge, which would raise an error otherwise.

In [13]:
# Add a lower case column of receptor_protein as common identifier between receptors and k12_proteome
receptors['receptor_protein_lower'] = receptors['receptor_protein'].str.lower()
k12_proteome['GN_lower'] = k12_proteome['GN_k12'].str.lower()

# Merge datasets
receptors_k12 = receptors.merge(k12_proteome, how='inner', left_on='receptor_protein_lower', right_on='GN_lower', suffixes=('_phage', '_k12'), validate='many_to_one')

# Drop GN_lower and receptor_protein_lower and order by bas
receptors_k12.drop(columns=['GN_lower', 'receptor_protein_lower'], inplace=True)
receptors_k12.sort_values(by=['bas'], inplace=True)

# Drop GN_k12 and GN_lower in k12_proteome
k12_proteome.drop(columns=['GN_k12', 'GN_lower'], inplace=True)

In [14]:
display(receptors_k12.head())
display(receptors_k12.shape)

,bas,phage,receptor_protein,seqID_k12,sequence_k12,GN_k12
0,Bas01,Escherichia phage AugustePiccard,LptD,sp|P31554|LPTD_ECOLI,MKKRIPTLLATMIATALYSQQGLAADLASQCMLGVPSYDRPLVQGD...,lptD
1,Bas02,Escherichia phage JeanPiccard,LptD,sp|P31554|LPTD_ECOLI,MKKRIPTLLATMIATALYSQQGLAADLASQCMLGVPSYDRPLVQGD...,lptD
7,Bas03,Escherichia phage JulesPiccard,FhuA,sp|P06971|FHUA_ECOLI,MARSKTAQPKHSLRKIAVVVATAVSGMSVYAQAAVEPKEDTITVTA...,fhuA
17,Bas04,Escherichia phage FritzSarasin,BtuB,sp|P06129|BTUB_ECOLI,MIKKASLLTACSVTAFSAWAQDTSPDTLVVTANRFEQPRSTVLAPT...,btuB
8,Bas05,Escherichia phage PeterMerian,FhuA,sp|P06971|FHUA_ECOLI,MARSKTAQPKHSLRKIAVVVATAVSGMSVYAQAAVEPKEDTITVTA...,fhuA


(50, 6)

Let's store a dict of the k12 proteins which are receptors involved in any interaction with the phages and the other way around

In [15]:
# Store seqID_k12 unique values k12_set['receptors'] the seqID_k12 of k12_proteome which are not in receptors_k12 in k12_set['others']
k12_set = {'receptors': set(receptors_k12['seqID_k12'].tolist()),
           'others': set(k12_proteome['seqID_k12'].tolist()) - set(receptors_k12['seqID_k12'].tolist())}

In [16]:
# Print the number of elements in each set
print('Number of receptors: {}'.format(len(k12_set['receptors'])))
print('Number of others: {}'.format(len(k12_set['others'])))

Number of receptors: 10
Number of others: 4393


## Fish RBPs
Let's define a regex to get RBPs from the 'protein' tag of basel proteome, as well a regex to exclude the ones which are not RBPs. \

In [17]:
# Define gene column in basel_proteome from the seqID_phage
# Where the seqID_phage has lcl|MZ501051.1_prot_QXV76132.1_1, the gene is MZ501051
basel_proteome['gene'] = basel_proteome['seqID_phage'].str.split('.').str[0].str.split('|').str[1]

In [19]:
import re
INCLUDE_REGEX = 'tail?(.?|\s*)(?:spike?|fib(?:er|re))|recept(?:o|e)r(.?|\s*)(?:bind|recogn).*(?:protein)?|(?<!\w)RBP(?!a)'
# Create exclusion list from the file in data/raw as a set. The file is in format gene1|gene2|gene3|...
# Reference:
# https://github.com/RyanCook94/inphared/blob/main/exclusion_list.txt
with open(os.path.join('..', 'data', 'raw', 'exclusion_list.txt'), 'r') as f: 
    EXCLUSION_LIST = set(f.read().split('|'))

In [21]:
# Filter basel_proteeme to keep only the rows where protein matches the regex
RBPs = basel_proteome.dropna(subset=['protein'])
RBPs = RBPs[RBPs['protein'].str.contains(INCLUDE_REGEX, flags=re.IGNORECASE, regex=True)]
print(f'Number of rows in RBPs: {len(RBPs)}')

# Remove rows where gene matches the exclusion list
RBPs = RBPs[~RBPs['gene'].isin(EXCLUSION_LIST)]
print(f'Number of rows in RBPs after removing exclusion list: {len(RBPs)}')

Number of rows in RBPs: 852
Number of rows in RBPs after removing exclusion list: 852


C:\Users\pf259\AppData\Local\Temp\ipykernel_18124\737712705.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  RBPs = RBPs[RBPs['protein'].str.contains(INCLUDE_REGEX, flags=re.IGNORECASE, regex=True)]
